# Segmentação de sinais

In [1]:
import pandas as pd
import numpy as np
import mne

In [8]:
infos_path = '/home/davi/Documentos/doutorado_ppgee_v2/data/siena_infos.json'
path_data = '/media/davi/6A81-05CF/physionet.org/files/siena-scalp-eeg/1.0.0/PN00/'

infos = pd.read_json(infos_path)

In [9]:
info = infos.iloc[0]
info

name                                                         PN00-1.edf
registration_start                                             19.39.33
registration_end                                               20.22.58
seizure_start                                                  19.58.36
seizure_end                                                    19.59.46
total                                                              2605
seizure_start_sec                                                  1392
seizure_end_sec                                                    1462
total_seizure                                                        70
highpass                                                       1.591549
lowpass                                                              30
meas_date                                                    2016-01-01
sfreq                                                               512
ch_names              [EEG Fp1, EEG F3, EEG C3, EEG P3, EEG O1, 

In [7]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Doutorado').getOrCreate()

In [10]:
d = spark.read.parquet(f"{path_data}{info['name'].replace('edf','parquet')}")

import numpy 